## NOTE - Use Python 3.6

In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
import pymongo
import dill
import os

from BrattEssay import load_bratt_essays
from load_data import load_process_essays
from window_based_tagger_config import get_config
from FindFiles import find_files
from DirUtils import dir_exists
from results_procesor import ResultsProcessor, __MICRO_F1__
from Settings import Settings
from statsmodels.stats.contingency_tables import mcnemar

In [2]:
FOLDER = "/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Predictions/"

## Find Closest Results

In [25]:
with open(FOLDER + "SENT_TEST_CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_FIXED_VD_YS_.dill", "rb+") as f:
    cb_ysbytag = dill.load(f)
    
with open(FOLDER + "SENT_TEST_CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM_VD_2_YS_.dill", "rb+") as f:
    sc_ysbytag = dill.load(f)

In [30]:
def filter_by_str(filters, files, exclude=False):
    if type(filters) != list:
        filters = [filters]
    flt = files
    for fltr in filters:
        files = [f for f in files if (fltr in f) != exclude]
    return  files

In [36]:
# FILTER = "STACKED"
FILTER = "SENT"

In [77]:
all_files = os.listdir(FOLDER)
# print(len(all_files))
perc_files = all_files
perc_files = filter_by_str("_CB_", perc_files)
# perc_files = filter_by_str("_CR_", perc_files)
perc_files = filter_by_str("TEST_", perc_files)
perc_files = filter_by_str("_VD_", perc_files)
perc_files = filter_by_str("_PREDS", perc_files)
perc_files = filter_by_str(FILTER + "_", perc_files)
# perc_files = filter_by_str("2019", perc_files)
print(len(perc_files))
perc_files

33


['SENT_TEST_CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_FIXED_VD_PREDS__2019-10-12_11-9-55.dill',
 'SENT_TEST_CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_FIXED_VD_PREDS__2019-10-12_10-47.dill',
 'SENT_TEST_CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_FIXED_VD_PREDS__2019-10-12_11-8-42.dill',
 'SENT_TEST_CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_FIXED_VD_PREDS__2019-10-12_10-50-27.dill',
 'SENT_TEST_CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_FIXED_VD_PREDS__2019-10-12_11-10-20.dill',
 'SENT_TEST_CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_FIXED_VD_PREDS__2019-10-12_10-46.dill',
 'SENT_TEST_CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_FIXED_VD_PREDS__2019-10-12_11-11-10.dill',
 'SENT_TEST_CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_FIXED_VD_PREDS__2019-10-12_10-41.dill',
 'SENT_TEST_CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_FIXED_VD_PREDS__2019-10-12_10-16.dill',
 'SENT_TEST_CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_FIXED_VD_PREDS__.dill',
 'SENT_TEST_CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_FIXED_VD_PREDS__2019-10-12_10-40.dill',
 'SENT_TEST_CR_CB_SHIFT_REDUCE_PARS

In [78]:
import pandas as pd

ysbytag = cb_ysbytag
f2preds = dict()
rows = []
for f in perc_files[::-1]:
    with open(FOLDER + f, "rb+") as fin:
        preds = dill.load(fin)
    metrics = ResultsProcessor.compute_mean_metrics(ysbytag, preds)[__MICRO_F1__]
    metrics["fname"] = f[-24:]
    rows.append(metrics)
df = pd.DataFrame(rows).sort_values("f1_score")
df # 0.837

,accuracy,data_points,f1_score,fname,num_codes,precision,recall
3,0.985121,20566.0,0.732049,__2019-10-12_11-9-7.dill,547.0,0.702521,0.764168
8,0.985316,20566.0,0.734622,S__2019-10-12_10-45.dill,547.0,0.707276,0.764168
6,0.985364,20566.0,0.735733,2019-10-12_11-12-24.dill,547.0,0.707770,0.765996
13,0.985413,20566.0,0.735915,2019-10-12_11-10-45.dill,547.0,0.709677,0.764168
23,0.985461,20566.0,0.736564,ED_FIXED_VD_PREDS__.dill,547.0,0.710884,0.764168
7,0.985461,20566.0,0.736564,2019-10-12_11-11-59.dill,547.0,0.710884,0.764168
0,0.985316,20566.0,0.737847,S__2019-10-12_10-13.dill,547.0,0.702479,0.776965
24,0.985510,20566.0,0.738137,S__2019-10-12_10-16.dill,547.0,0.710660,0.767824
25,0.985559,20566.0,0.738786,S__2019-10-12_10-41.dill,547.0,0.711864,0.767824
11,0.985607,20566.0,0.738977,_2019-10-12_11-9-31.dill,547.0,0.713799,0.765996


## SC 

In [79]:
all_files = os.listdir(FOLDER)
perc_files = all_files
perc_files = filter_by_str("_SC_", perc_files)
perc_files = filter_by_str("_VD_", perc_files)
perc_files = filter_by_str("TEST_", perc_files)
perc_files = filter_by_str("_PREDS_", perc_files)
perc_files = filter_by_str(FILTER + "_", perc_files)
# perc_files = filter_by_str("_2019", perc_files)
perc_files

['SENT_TEST_CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM_VD_2_PREDS__2019-10-12_10-16.dill',
 'SENT_TEST_CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM_VD_2_PREDS__2019-10-12_10-20.dill',
 'SENT_TEST_CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM_VD_2_PREDS__2019-10-12_10-17.dill',
 'SENT_TEST_CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM_VD_2_PREDS_.dill',
 'SENT_TEST_CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM_VD_2_PREDS__2019-10-12_10-13.dill',
 'SENT_TEST_CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM_VD_2_PREDS__2019-10-12_10-14.dill',
 'SENT_TEST_CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM_VD_2_PREDS__2019-10-12_10-18.dill',
 'SENT_TEST_CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM_VD_2_PREDS__2019-10-12_10-19.dill',
 'SENT_TEST_CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM_VD_2_PREDS__2019-10-12_10-15.dill']

In [81]:
import pandas as pd

ysbytag = sc_ysbytag
f2preds = dict()
rows = []
for f in perc_files:
    with open(FOLDER + f, "rb+") as fin:
        preds = dill.load(fin)
    metrics = ResultsProcessor.compute_mean_metrics(ysbytag, preds)[__MICRO_F1__]
    metrics["fname"] = f[-16:]
    rows.append(metrics)
df = pd.DataFrame(rows).sort_values("f1_score")
df # 0.837

,accuracy,data_points,f1_score,fname,num_codes,precision,recall
2,0.973095,11336.0,0.826606,10-12_10-17.dill,910.0,0.856302,0.798901
1,0.973183,11336.0,0.827076,10-12_10-20.dill,910.0,0.857311,0.798901
3,0.973183,11336.0,0.827076,VD_2_PREDS_.dill,910.0,0.857311,0.798901
5,0.973271,11336.0,0.828523,10-12_10-14.dill,910.0,0.854142,0.804396
7,0.973536,11336.0,0.828571,10-12_10-19.dill,910.0,0.863095,0.796703
4,0.973359,11336.0,0.828604,10-12_10-13.dill,910.0,0.856808,0.802198
8,0.973536,11336.0,0.829739,10-12_10-15.dill,910.0,0.857981,0.803297
6,0.973800,11336.0,0.830962,10-12_10-18.dill,910.0,0.861865,0.802198
0,0.973712,11336.0,0.831257,10-12_10-16.dill,910.0,0.857477,0.806593
